# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [209]:
import pandas as pd

us_data_url = './data/US_interest_rates.csv'
IL_data_url = './data/BR.csv'
exchange_url = './data/EXR.csv'

 נביא את שתי המאגר מידע על הריבית לאותו פורמט

In [185]:
il_df = pd.read_csv(IL_data_url)
il_df = il_df.loc[:, ['TIME_PERIOD', 'OBS_VALUE']]
il_df = il_df.rename(columns={"TIME_PERIOD": "date", "OBS_VALUE": "rate"})
il_df.head()

,date,rate
0,1994-01-27,10.5
1,1994-01-28,10.5
2,1994-01-29,10.5
3,1994-01-30,10.5
4,1994-01-31,10.5


In [186]:
us_df = pd.read_csv(us_data_url)
us_df.head()

,date,rate
0,1954-07-01,1.13
1,1954-07-02,1.25
2,1954-07-03,1.25
3,1954-07-04,1.25
4,1954-07-05,0.88


In [187]:
def get_relevant_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.loc[(df['date'].dt.year > 1993) & (df['date'].dt.day == 1) & (df['date'].dt.year < 2024)]
    return df

In [188]:
def calculate_change(current_rate):
    Change_From_Last_Month = []
    prv_rate = current_rate.iloc[0]
    for rate in current_rate:
        change = rate - prv_rate
        Change_From_Last_Month.append(change)
        prv_rate = rate
    return Change_From_Last_Month

In [194]:
def crate_new_df(df):
    years = df['date'].dt.year  
    months = df['date'].dt.month
    current_rate = df.loc[ :,'rate']
    Change_From_Last_Month = calculate_change(current_rate)
    data = {
    "year": years,
    "month": months,
    "current_rate": current_rate,
    "change_from_last_month": Change_From_Last_Month
    }
    return pd.DataFrame(data)
    

In [198]:
il_mew_df = crate_new_df(get_relevant_data(il_df))
us_mew_df = crate_new_df(get_relevant_data(us_df))
us_mew_df

,year,month,current_rate,change_from_last_month
14429,1994,1,2.85,0.00
14460,1994,2,3.25,0.40
14488,1994,3,3.31,0.06
14519,1994,4,3.64,0.33
14549,1994,5,3.73,0.09
...,...,...,...,...
24601,2023,6,5.08,0.25
24642,2023,8,5.33,0.25
24665,2023,9,5.33,0.00
24706,2023,11,5.33,0.00


In [214]:
exchange_df = pd.read_csv(exchange_url)
exchange_df = exchange_df.loc[:, ['TIME_PERIOD', 'OBS_VALUE']]
exchange_df = exchange_df.rename(columns={"TIME_PERIOD": "date", "OBS_VALUE": "rate"})

exchange_df.head()
exchange_mew_df = crate_new_df(get_relevant_data(exchange_df))
exchange_mew_df

,year,month,current_rate,change_from_last_month
26,1994,2,2.976,0.000
49,1994,3,2.965,-0.011
97,1994,5,3.000,0.035
123,1994,6,3.035,0.035
149,1994,7,3.035,0.000
...,...,...,...,...
7342,2023,6,3.736,0.117
7384,2023,8,3.653,-0.083
7407,2023,9,3.795,0.142
7448,2023,11,4.029,0.234


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [200]:
# A - 1
# your code here
us_mew_df.head()

In [205]:
# A - 2
# your code here
average = il_mew_df['current_rate'].mean()
average

4.879108635097493

In [215]:
# A - 3
# your code here
highest_change = exchange_mew_df['change_from_last_month'].max()
highest_change

0.37200000000000033

In [219]:
# A - 4
# your code here
total_decreased_months = (us_mew_df['change_from_last_month'] < 0).sum()
total_decreased_months

140

In [228]:
# A - 5
# your code here
def calculate_Change_Percentage(df):
    changes = exchange_mew_df['current_rate']
    Change_Percentage = []
    prv_change = changes.iloc[0]
    for c in changes:
        if prv_change == 0:
            prv_change = 0.001
        change = (c*100)/prv_change
        Change_Percentage.append(change)
        prv_rate = c
    return Change_Percentage

Change_Percentage = calculate_Change_Percentage(exchange_mew_df)
exchange_mew_df.insert(4, 'Change_Percentage', Change_Percentage)
exchange_mew_df.head()

,year,month,current_rate,change_from_last_month,Change_Percentage
26,1994,2,2.976,0.000,100.000000
49,1994,3,2.965,-0.011,99.630376
97,1994,5,3.000,0.035,100.806452
123,1994,6,3.035,0.035,101.982527
149,1994,7,3.035,0.000,101.982527


## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [1]:
# B - 1
# your code here

In [2]:
# B - 2
# your code here

In [3]:
# B - 3
# your code here

In [4]:
# B - 4
# your code here

In [6]:
# B - 5
# your code here

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here